# Multi-Style Text Generator with Emotion Sliders

This notebook implements a multi-style text generator with emotion sliders using a pre-trained transformer (GPT-2) augmented with LoRA for style conditioning and soft prompts for emotion control. The system allows users to generate text (e.g., reviews, responses) with adjustable styles (formal, casual) and emotions (joy, sadness).

## Setup
- **Model**: GPT-2 Small (117M parameters) for efficiency.
- **Libraries**: Hugging transformers, peft (for LoRA), torch.
- **Functionality**: Users select a style via LoRA adapters and adjust emotion sliders to modulate output.

In [1]:
!pip install torch transformers peftimport torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from peft import LoraConfig, get_peft_model
import numpy as np

# Initialize model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Define LoRA for style conditioning (formal, casual)
lora_config = LoraConfig(
    r=8,  # Rank of LoRA matrices
    lora_alpha=32,
    target_modules=['c_attn', 'c_proj'],
    lora_dropout=0.1,
    bias='none'
)
model = get_peft_model(model, lora_config)

# Simulated LoRA weights (in practice, train these)
style_weights = {
    'formal': torch.randn(model.config.n_embd, 8),  # Dummy weights
    'casual': torch.randn(model.config.n_embd, 8)
}

# Soft prompt for emotion control (joy, sadness)
class SoftPrompt(torch.nn.Module):
    def __init__(self, embed_size, prompt_length=10):
        super().__init__()
        self.prompt = torch.nn.Parameter(torch.randn(prompt_length, embed_size))
    def forward(self):
        return self.prompt

# Initialize soft prompts for emotions
embed_size = model.config.n_embd
soft_prompt_joy = SoftPrompt(embed_size)
soft_prompt_sadness = SoftPrompt(embed_size)

# Function to combine emotions based on slider values
def get_emotion_prompt(joy_val, sadness_val):
    # Normalize slider values (-1 to 1)
    joy_val = max(min(joy_val, 1), -1)
    sadness_val = max(min(sadness_val, 1), -1)
    # Weighted combination
    weight_joy = (joy_val + 1) / 2  # Map to [0,1]
    weight_sadness = (sadness_val + 1) / 2
    total = weight_joy + weight_sadness
    if total == 0:
        total = 1  # Avoid division by zero
    combined_prompt = (weight_joy * soft_prompt_joy() + weight_sadness * soft_prompt_sadness()) / total
    return combined_prompt

# Generation function
def generate_text(prompt, style='formal', joy_val=0.0, sadness_val=0.0, max_length=50):
    # Set LoRA weights for style (simulated)
    # In practice, load trained LoRA adapters
    # Apply style-specific LoRA (placeholder)
    inputs = tokenizer(prompt, return_tensors='pt')
    input_ids = inputs['input_ids']
    
    # Get emotion soft prompt
    emotion_prompt = get_emotion_prompt(joy_val, sadness_val)
    
    # Concatenate soft prompt embeddings with input
    embeddings = model.get_input_embeddings()(input_ids)
    prompt_embeds = torch.cat([emotion_prompt.unsqueeze(0), embeddings], dim=1)
    
    # Generate
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            inputs_embeds=prompt_embeds,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = 'Write a review for a smartphone.'
print('Formal, Joyful Review:')
print(generate_text(prompt, style='formal', joy_val=0.8, sadness_val=-0.2))
print('\nCasual, Sad Review:')
print(generate_text(prompt, style='casual', joy_val=-0.5, sadness_val=0.7))

## Notes
- **Training**: In a real implementation, train LoRA adapters on style-specific datasets (e.g., formal academic texts, casual social media posts) and soft prompts on emotion-labeled data (e.g., GoEmotions).
- **UI Integration**: For sliders, deploy via a web app (e.g., Flask + HTML sliders) calling `generate_text`.
- **Limitations**: Current code uses dummy LoRA weights; actual training required for production.